# MP4文件格式概览

MP4文件由多个box组成，每个box存储不同的信息，且box之间是树状结构，如下图所示。

![](https://p3-juejin.byteimg.com/tos-cn-i-k3u1fbpfcp/be249051d03f45748b3055c842d75793~tplv-k3u1fbpfcp-watermark.image)


# MP4 Box简介
1个box由两部分组成：box header、box body。

- box header：box的元数据，比如box type、box size。
- box body：box的数据部分，实际存储的内容跟box类型有关，比如mdat中body部分存储的媒体数据。

box header中，只有type、size是必选字段。
当size==0时，存在largesize字段。
在部分box中，还存在version、flags字段，这样的box叫做Full Box。
当box body中嵌套其他box时，这样的box叫做container box。

![](https://p3-juejin.byteimg.com/tos-cn-i-k3u1fbpfcp/f6b3386f5de345cc9688a6f5e968ae28~tplv-k3u1fbpfcp-watermark.image)

## Box Header
字段定义如下：

- type：box类型，包括 “预定义类型”、“自定义扩展类型”，占4个字节；
    - 预定义类型：比如ftyp、moov、mdat等预定义好的类型；
    - 自定义扩展类型：如果type==uuid，则表示是自定义扩展类型。size（或largesize）随后的16字节，为自定义类型的值（extended_type）
- size：包含box header在内的整个box的大小，单位是字节。当size为0或1时，需要特殊处理：
    - size等于0：box的大小由后续的largesize确定（一般只有装载媒体数据的mdat box会用到largesize）；
    - size等于1：当前box为文件的最后一个box，通常包含在mdat box中；
- largesize：box的大小，占8个字节；
- extended_type：自定义扩展类型，占16个字节；

In [57]:
from construct import *

f = open("./data/data1.mp4", "rb")
data = f.read()
f.close()


class Box:
    def __init__(self):
        if not hasattr(self, 'body_fmt'):
            self.body_fmt = IfThenElse(
                this.size == 1,
                GreedyBytes,
                Bytes(
                    this.size - 8 if this.large_size == None else this.large_size - 16
                ),
            )

        self.struct = None
        self.size = None
        self.box_type = None
        self.body = None

        self.fmt = Struct(
            "size" / Int32ub,
            "type" / PaddedString(4, "ascii"),
            "large_size" / Optional(If(this.size == 0, Int64ub)),
            "extended_type" / Optional(If(this.type == "uuid", Bytes(16))),
            "body" / self.body_fmt,
        )

    def __str__(self):
        ret = f"box_type: {self.box_type}\nNormal Box " + self.struct.__str__()
        return ret

    def init(self, data):
        self.struct = self.fmt.parse(data)
        self.size = (
            self.struct.size
            if self.struct.large_size == None
            else self.struct.large_size
        )
        self.box_type = self.struct.type
        self.body = self.struct.body

    @staticmethod
    def getBoxList(data):
        cur_offset = 0
        boxes = []
        data_len = len(data)

        while cur_offset < data_len:
            tmp = Box(data[cur_offset:])

            # print(f"Current pos is {cur_offset}/{data_len}")

            box = None
            if tmp.box_type == "avc1":
                box = VisualSampleEntry(data[cur_offset:])
            else:
                box = tmp

            boxes.append(box)
            cur_offset += box.size

        return boxes


box = Box()
box.init(data)
print(box)


box_type: ftyp
Normal Box Container: 
    size = 32
    type = u'ftyp' (total 4)
    large_size = None
    extended_type = None
    body = b'isom\x00\x00\x02\x00isomiso2'... (truncated, total 24)


## FullBox
在Box的基础上，扩展出了FullBox类型。相比Box，FullBox 多了 version、flags 字段。

- version：当前box的版本，为扩展做准备，占1个字节；
- flags：标志位，占24位，含义由具体的box自己定义；

FullBox主要在moov中的box用到，比如 moov.mvhd

In [58]:
class FullBox(Box):
    def __init__(self):
        if not hasattr(self, 'body_fmt'):
            self.body_fmt = IfThenElse(
                this.size == 1,
                GreedyBytes,
                Bytes(
                    this.size - 12 if this.large_size == None else this.large_size - 20
                ),
            )

        self.struct = None
        self.size = None
        self.box_type = None
        self.body = None
        self.version = None
        self.flags = None
        self.body = None

        self.fmt = Struct(
            "size" / Int32ub,
            "type" / PaddedString(4, "ascii"),
            "large_size" / Optional(If(this.size == 0, Int64ub)),
            "extended_type" / Optional(If(this.type == "uuid", Bytes(16))),
            "version" / Int8ub,
            "flags" / Bytes(3),
            "body" / self.body_fmt,
        )

    def init(self, data):
        super().init(data)
        self.version = self.struct.version
        self.flags = self.struct.flags

    def __str__(self):
        ret = f"box_type: {self.box_type}\nFull Box " + self.struct.__str__()
        return ret


# ftyp(File Type Box)
ftyp用来指出当前文件遵循的规范

## 什么是isom
isom（ISO Base Media file）是在 MPEG-4 Part 12 中定义的一种基础文件格式，MP4、3gp、QT 等常见的封装格式，都是基于这种基础文件格式衍生的。

MP4 文件可能遵循的规范有mp41、mp42，而mp41、mp42又是基于isom衍生出来的。

>3gp(3GPP)：一种容器格式，主要用于3G手机上；  
>QT：QuickTime的缩写，.qt 文件代表苹果QuickTime媒体文件；

## ftyp 定义
其头部信息和 Box 一致, 主要是 body 部分:
- major_brand(4 字节字符串): 比如常见的 isom、mp41、mp42、avc1、qt等。它表示“最好”基于哪种格式来解析当前的文件。
- minor_version(4 字节无符号整数): 提供 major_brand 的说明信息，比如版本号，不得用来判断媒体文件是否符合某个标准/规范
- compatible_brands(4 字节字符串列表): 文件兼容的brand列表。比如 mp41 的兼容 brand 为 isom。通过兼容列表里的 brand 规范，可以将文件 部分（或全部）解码出来；

>在实际使用中，不能把 isom 做为 major_brand，而是需要使用具体的brand（比如mp41），  
>因此，对于 isom，没有定义具体的文件扩展名、mime type。

下面是常见的几种brand，以及对应的文件扩展名、mime type，更多brand可以参考 [这里](http://fileformats.archiveteam.org/wiki/Boxes/atoms_format#Brands) 

![](https://p9-juejin.byteimg.com/tos-cn-i-k3u1fbpfcp/9290f510d66a45699e0c6c9d3b32562e~tplv-k3u1fbpfcp-watermark.image)



In [59]:
class FileTypeBox(Box):
    def __init__(self):
        self.body_fmt = Struct(
            "major_brand" / PaddedString(4, "ascii"),
            "minor_version" / Int32ub,
            "compatible_brands" / GreedyRange(PaddedString(4, "ascii")),
        )

        super().__init__()


ftyp = FileTypeBox()
ftyp.init(data)
print(ftyp)


box_type: ftyp
Normal Box Container: 
    size = 32
    type = u'ftyp' (total 4)
    large_size = None
    extended_type = None
    body = Container: 
        major_brand = u'isom' (total 4)
        minor_version = 512
        compatible_brands = ListContainer: 
            isom
            iso2
            avc1
            mp41


# moov(Movie Box)

Movie Box，存储 mp4 的 metadata，一般位于mp4文件的开头。

moov中，最重要的两个box是 mvhd 和 trak：

- mvhd：Movie Header Box，mp4文件的整体信息，比如创建时间、文件时长等；
- trak：Track Box，一个mp4可以包含一个或多个轨道（比如视频轨道、音频轨道），轨道相关的信息就在trak里。trak是container box，至少包含两个box，tkhd、mdia；

>mvhd针对整个影片，tkhd针对单个track，mdhd针对媒体，vmhd针对视频，smhd针对音频，  
>可以认为是从 宽泛 > 具体，前者一般是从后者推导出来的。

## mvhd(Movie Header Box)

MP4文件的整体信息，跟具体的视频流、音频流无关，比如创建时间、文件时长等。

定义如下:
```java
aligned(8) class MovieHeaderBox extends FullBox(‘mvhd’, version, 0) { 
    if (version==1) {
      unsigned int(64)  creation_time;
      unsigned int(64)  modification_time;
      unsigned int(32)  timescale;
      unsigned int(64)  duration;
    } else { // version==0
      unsigned int(32)  creation_time;
      unsigned int(32)  modification_time;
      unsigned int(32)  timescale;
      unsigned int(32)  duration;
    }
    template int(32) rate = 0x00010000; // typically 1.0
    template int(16) volume = 0x0100; // typically, full volume 
    const bit(16) reserved = 0;
    const unsigned int(32)[2] reserved = 0;
    template int(32)[9] matrix =
        { 0x00010000,0,0,0,0x00010000,0,0,0,0x40000000 };
        // Unity matrix
    bit(32)[6]  pre_defined = 0;
    unsigned int(32)  next_track_ID;
}
```
含义如下:

- creation_time：文件创建时间；
- modification_time：文件修改时间；
- timescale：一秒包含的时间单位（整数）。举个例子，如果timescale等于1000，那么，一秒包含1000个时间单位（后面track等的时间，都要用这个来换算，比如track的duration为10,000，那么，track的实际时长为10,000/1000=10s）；
- duration：影片时长（整数），根据文件中的track的信息推导出来，等于时间最长的track的duration；
- rate：推荐的播放速率，32位整数，高16位、低16位分别代表整数部分、小数部分（[16.16]），举例 0x0001 0000 代表1.0，正常播放速度；
- volume：播放音量，16位整数，高8位、低8位分别代表整数部分、小数部分（[8.8]），举例 0x01 00 表示 1.0，即最大音量；
- matrix：视频的转换矩阵，一般可以忽略不计；
- next_track_ID：32位整数，非0，一般可以忽略不计。当要添加一个新的track到这个影片时，可以使用的track id，必须比当前已经使用的track id要大。也就是说，添加新的track时，需要遍历所有track，确认可用的track id；

In [60]:
class MovieHeaderBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "creation_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "modification_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "timescale" / Int32ub,
            "duration" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "rate" / Bytes(4),
            "volume" / Bytes(2),
            "reserved1" / Bytes(2),
            "reserved2" / Bytes(8),
            "matrix" / Bytes(4)[9],
            "pre_defined" / Bytes(4)[6],
            "next_track_ID" / Int32ub,
        )

        super().__init__()

    def init(self, data):
        super().init(data)
        self.body.rate = self.convertRate(self.body.rate)
        self.body.volume = self.convertVolume(self.body.volume)

    def convertRate(self, rate_byte):
        high = int.from_bytes(rate_byte[:2], byteorder="big")
        low = int.from_bytes(rate_byte[2:], byteorder="big")
        return str(high) + "." + str(low)

    def convertVolume(self, volume_byte):
        high = int.from_bytes(volume_byte[:1], byteorder="big")
        low = int.from_bytes(volume_byte[1:], byteorder="big")
        return str(high) + "." + str(low)


mvhd_offset = data.find(b"mvhd") - 4
mvhd = MovieHeaderBox()
mvhd.init(data[mvhd_offset:])
# print(mvhd.body)
print(mvhd)


box_type: mvhd
Full Box Container: 
    size = 108
    type = u'mvhd' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x00' (total 3)
    body = Container: 
        creation_time = 0
        modification_time = 0
        timescale = 1000
        duration = 134768
        rate = u'1.0' (total 3)
        volume = u'1.0' (total 3)
        reserved1 = b'\x00\x00' (total 2)
        reserved2 = b'\x00\x00\x00\x00\x00\x00\x00\x00' (total 8)
        matrix = ListContainer: 
            b'\x00\x01\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x01\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'@\x00\x00\x00'
        pre_defined = ListContainer: 
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'

## trak(Track Box)

Track Box 是一个 Container Box, 里面包含至少两个 Box, tkhd(Track Header Box), media
一个mp4可以包含一个或多个轨道（比如视频轨道、音频轨道），轨道相关的信息就在 trak 里。

定义如下: 
```java
aligned(8) class TrackBox extends Box(‘trak’) { 
} 
```

## thkd(Track Header Box)

定义如下:

```java
aligned(8) class TrackHeaderBox 
  extends FullBox(‘tkhd’, version, flags){ 
	if (version==1) {
	      unsigned int(64)  creation_time;
	      unsigned int(64)  modification_time;
	      unsigned int(32)  track_ID;
	      const unsigned int(32)  reserved = 0;
	      unsigned int(64)  duration;
	   } else { // version==0
	      unsigned int(32)  creation_time;
	      unsigned int(32)  modification_time;
	      unsigned int(32)  track_ID;
	      const unsigned int(32)  reserved = 0;
	      unsigned int(32)  duration;
	}
	const unsigned int(32)[2] reserved = 0;
	template int(16) layer = 0;
	template int(16) alternate_group = 0;
	template int(16) volume = {if track_is_audio 0x0100 else 0}; 
    const unsigned int(16) reserved = 0;
	template int(32)[9] matrix= { 0x00010000,0,0,0,0x00010000,0,0,0,0x40000000 }; // unity matrix
	unsigned int(32) width;
	unsigned int(32) height;
}
```

单个 track 的 metadata，包含如下字段：

- version：tkhd box的版本；
- flags：按位或操作获得，默认值是7（0x000001 | 0x000002 | 0x000004），表示这个track是启用的、用于播放的 且 用于预览的。
    - Track_enabled：值为0x000001，表示这个track是启用的，当值为0x000000，表示这个track没有启用；
    - Track_in_movie：值为0x000002，表示当前track在播放时会用到；
    - Track_in_preview：值为0x000004，表示当前track用于预览模式；
- creation_time：当前track的创建时间；
- modification_time：当前track的最近修改时间；
- track_ID：当前track的唯一标识，不能为0，不能重复；
- duration：当前track的完整时长（需要除以timescale得到具体秒数）；
- layer：视频轨道的叠加顺序，数字越小越靠近观看者，比如1比2靠上，0比1靠上；
- alternate_group：当前track的分组ID，alternate_group值相同的track在同一个分组里面。同个分组里的track，同一时间只能有一个track处于播放状态。当alternate_group为0时，表示当前track没有跟其他track处于同个分组。一个分组里面，也可以只有一个track；
- volume：audio track的音量，介于0.0~1.0之间；
- matrix：视频的变换矩阵；
- width、height：视频的宽高；

In [78]:
class TrackBox(Box):
    def __init__(self):
        super().__init__()


class TrackHeaderBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "creation_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "modification_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "track_ID" / Int32ub,
            "reserved1" / Int32ub,
            "duration" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "reserved2" / Bytes(8),
            "layer" / Int16sb,
            "alternate_group" / Int16sb,
            "volume" / Bytes(2),
            "reserved3" / Bytes(2),
            "matrix" / Bytes(4)[9],
            "width" / Aligned(4, Int16ub),
            "height" / Aligned(4, Int16ub),
        )

        super().__init__()

    def init(self, data):
        super().init(data)

        self.body.volume = self.convertVolume(self.body.volume)

    def convertVolume(self, volume_byte):
        high = int.from_bytes(volume_byte[:1], byteorder="big")
        low = int.from_bytes(volume_byte[1:], byteorder="big")
        return str(high) + "." + str(low)


trak_offset = data.find(b"trak") - 4
trak = TrackBox()
trak.init(data[trak_offset:])
print(trak)
print()
thkd = TrackHeaderBox()
thkd.init(trak.body)
# print(thkd.body)
print(thkd)


box_type: trak
Normal Box Container: 
    size = 54678
    type = u'trak' (total 4)
    large_size = None
    extended_type = None
    body = b'\x00\x00\x00\\tkhd\x00\x00\x00\x03\x00\x00\x00\x00'... (truncated, total 54670)

box_type: tkhd
Full Box Container: 
    size = 92
    type = u'tkhd' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x03' (total 3)
    body = Container: 
        creation_time = 0
        modification_time = 0
        track_ID = 1
        reserved1 = 0
        duration = 134668
        reserved2 = b'\x00\x00\x00\x00\x00\x00\x00\x00' (total 8)
        layer = 0
        alternate_group = 0
        volume = u'0.0' (total 3)
        reserved3 = b'\x00\x00' (total 2)
        matrix = ListContainer: 
            b'\x00\x01\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x01\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
  

## hdlr(Handler Reference Box)

声明当前track的类型，以及对应的处理器（handler）。

定义如下:
```java
aligned(8) class HandlerBox extends FullBox(‘hdlr’, version = 0, 0) { 
	unsigned int(32) pre_defined = 0;
	unsigned int(32) handler_type;
	const unsigned int(32)[3] reserved = 0;
   	string   name;
}
```

字段含义如下:
- handler_type: 有如下类型
    - vide（0x76 69 64 65），video track；
    - soun（0x73 6f 75 6e），audio track；
    - hint（0x68 69 6e 74），hint track；
- name: 为utf8字符串，对handler进行描述，比如 L-SMASH Video Handler（参考 [这里](http://avisynth.nl/index.php/LSMASHSource)）。

In [70]:
class HandlerBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "pre_defined" / Int32ub,
            "handler_type" / Bytes(4),
            "reserved" / Bytes(12),
            "name" / CString('utf8')
        )

        super().__init__()


hdlr_offset = data.find(b"hdlr") - 4
hdlr = HandlerBox()
hdlr.init(data[hdlr_offset:])
print(hdlr)


box_type: hdlr
Full Box Container: 
    size = 45
    type = u'hdlr' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x00' (total 3)
    body = Container: 
        pre_defined = 0
        handler_type = b'vide' (total 4)
        reserved = b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' (total 12)
        name = u'VideoHandler' (total 12)


## stbl(Sample Table Box)

MP4文件的媒体数据部分在mdat box里，而stbl则包含了这些媒体数据的索引以及时间信息，了解stbl对解码、渲染MP4文件很关键。

定义如下:
```java
aligned(8) class SampleTableBox extends Box(‘stbl’) { 
}
```

在MP4文件中，媒体数据被分成多个chunk，每个chunk可包含多个sample，而sample则由帧组成（通常1个sample对应1个帧），关系如下：

![](https://pic3.zhimg.com/80/v2-095434752cc54e67021a1f80bf1ba30e_720w.jpg)

stbl中比较关键的box包含stsd、stco、stsc、stsz、stts、stss、ctts。下面先来个概要的介绍，然后再逐个讲解细节。

### stco / stsc / stsz / stts / stss / ctts / stsd 概述
下面是这几个box概要的介绍：

- stsd：给出视频、音频的编码、宽高、音量等信息，以及每个sample中包含多少个frame；
- stco：thunk在文件中的偏移；
- stsc：每个thunk中包含几个sample；
- stsz：每个sample的size（单位是字节）；
- stts：每个sample的时长；
- stss：哪些sample是关键帧；
- ctts：帧解码到渲染的时间差值，通常用在B帧的场景；


### stsd（Sample Description Box）

stsd给出sample的描述信息，这里面包含了在解码阶段需要用到的任意初始化信息，比如 编码 等。对于视频、音频来说，所需要的初始化信息不同，这里以视频为例。

定义如下:
```java
aligned(8) abstract class SampleEntry (unsigned int(32) format) 
 extends Box(format){ 
 const unsigned int(8)[6] reserved = 0; 
 unsigned int(16) data_reference_index; 
} 
 
class HintSampleEntry() extends SampleEntry (protocol) { 
 unsigned int(8) data []; 
} 
 // Visual Sequences 
class VisualSampleEntry(codingname) extends SampleEntry (codingname){ 
 unsigned int(16) pre_defined = 0; 
 const unsigned int(16) reserved = 0; 
 unsigned int(32)[3]  pre_defined = 0; 
 unsigned int(16)  width; 
 unsigned int(16)  height; 
 template unsigned int(32)  horizresolution = 0x00480000; // 72 dpi 
 template unsigned int(32)  vertresolution  = 0x00480000; // 72 dpi 
 const unsigned int(32)  reserved = 0; 
 template unsigned int(16)  frame_count = 1; 
 string[32]  compressorname; 
 template unsigned int(16)  depth = 0x0018; 
 int(16)  pre_defined = -1; 
} 
 
 // Audio Sequences 
 
class AudioSampleEntry(codingname) extends SampleEntry (codingname){ 
 const unsigned int(32)[2] reserved = 0; 
 template unsigned int(16) channelcount = 2; 
 template unsigned int(16) samplesize = 16; 
 unsigned int(16) pre_defined = 0; 
 const unsigned int(16) reserved = 0 ; 
 template unsigned int(32) samplerate = {timescale of media}<<16; 
}

aligned(8) class SampleDescriptionBox (unsigned int(32) handler_type) 
 extends FullBox(’stsd’, 0, 0){ 
 int i ; 
 unsigned int(32) entry_count; 
 for (i = 1 ; i u entry_count ; i++){ 
  switch (handler_type){ 
   case ‘soun’: // for audio tracks 
    AudioSampleEntry(); 
    break; 
   case ‘vide’: // for video tracks  
    VisualSampleEntry(); 
    break; 
   case ‘hint’: // Hint track 
    HintSampleEntry(); 
    break; 
  } 
 } 
}
```



In [87]:
class AudioSampleEntry(Box):
    def __init__(self):
        self.body_fmt = Struct(
            "reserved" / Bytes(6),
            "data_reference_index" / Int16ub,
            "reserved1" / Bytes(8),
            "channelcount" / Int16ub,
            "samplesize" / Int16ub,
            "pre_defined" / Int16ub,
            "perserved2" / Bytes(2),
            "samplerate" / Aligned(4, Int16ub),
        )

        super().__init__()


class HintSampleEntry(Box):
    def __init__(self):
        self.body_fmt = Struct(
            "reserved" / Bytes(6),
            "data_reference_index" / Int16ub,
            "data" / Int8ub,
        )

        super().__init__()


class VisualSampleEntry(Box):
    def __init__(self):
        self.body_fmt = Struct(
            "reserved" / Bytes(6),
            "data_reference_index" / Int16ub,
            "pre_defined1" / Bytes(2),
            "reserved1" / Bytes(2),
            "pre_defined2" / Bytes(12),
            "width" / Int16ub,
            "height" / Int16ub,
            "horizresolution" / Aligned(4, Int16ub),
            "vertresolution" / Aligned(4, Int16ub),
            "reserved2" / Bytes(4),
            "frame_count" / Int16ub,
            "compressorname" / PaddedString(32, "utf8"),
            "depth" / Int16ub,
            "pre_defined3" / Bytes(2),
        )

        super().__init__()


class SampleDescriptionBox(FullBox):
    def __init__(self, handler_type):

        if handler_type == "vide":
            self.entry_fmt = VisualSampleEntry().fmt
        elif handler_type == "soun":
            self.entry_fmt = AudioSampleEntry().fmt
        elif handler_type == "hint":
            self.entry_fmt = HintSampleEntry().fmt

        self.body_fmt = Struct(
            "entry_count" / Int32ub, "entries" / Array(this.entry_count, self.entry_fmt)
        )

        super().__init__()


vide_stsd_offset = data.find(b"stsd") - 4
vide_stsd = SampleDescriptionBox("vide")
vide_stsd.init(data[vide_stsd_offset:])
print(vide_stsd)

soun_stsd_offset = data.find(b'stsd', vide_stsd_offset + 5) - 4
soun_stsd = SampleDescriptionBox('soun')
soun_stsd.init(data[soun_stsd_offset:])
print(soun_stsd)


box_type: stsd
Full Box Container: 
    size = 169
    type = u'stsd' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x00' (total 3)
    body = Container: 
        entry_count = 1
        entries = ListContainer: 
            Container: 
                size = 153
                type = u'avc1' (total 4)
                large_size = None
                extended_type = None
                body = Container: 
                    reserved = b'\x00\x00\x00\x00\x00\x00' (total 6)
                    data_reference_index = 1
                    pre_defined1 = b'\x00\x00' (total 2)
                    reserved1 = b'\x00\x00' (total 2)
                    pre_defined2 = b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' (total 12)
                    width = 640
                    height = 360
                    horizresolution = 72
                    vertresolution = 72
                    reserved2 = b'\x00\x00\x00\x00' (total 4)
         

In [ ]:
class MovieBox(Box):
    def __init__(self, data):
        super().__init__(data)


moov = MovieBox(data[ftyp.size :])
print(moov)


box_type: moov
Base Container: 
    size = 114726
    type = u'moov' (total 4)
    large_size = None
    extended_type = None
    body = b'\x00\x00\x00lmvhd\x00\x00\x00\x00\x00\x00\x00\x00'... (truncated, total 114718)
